## Churn prediction of bank customers using Artificial Neural Network

The goal of this model would be to predict if a customer will churn given recorded customer attributes like balance, products, tenure etc.

### Preprocessing

#### Libraries required for data processing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#### Dataset

Source: [Kaggle public dataset](https://www.kaggle.com/datasets/barelydedicated/bank-customer-churn-modeling
"Kaggle")





In [7]:
dataset = pd.read_csv(r'Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### Selecting features and target

As RowNumber, CustomerId, Surname will not help in training, using other features for predictors X and Exited for target y

In [8]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

#### Encoding categorical data

Converting the categorical data to numerical to be able to used as input to the model

##### Libraries for encoding

In [21]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

##### Encoding Geography

In [17]:
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

##### Encoding Gender

In [18]:
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

##### One hot encoding

Non-binary categorical variables require one hot encoding. here the column with index 1, (Geography) required one hot encoding

In [23]:
transformer = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(transformer.fit_transform(X))
print(X[0])

[1.0 0.0 1.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]


#### Splitting data
Splitting the dataset into the Training and Test set using a test size of 20%

##### Libraries for splitting

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#### Scaling
Scaling the data is necessary to minimize the effects of the differences between the numerical range of various features

##### Libraries for scaling

In [33]:
from sklearn.preprocessing import StandardScaler

##### Standard scaling
The scaling parameters are calculated using X_train to transform both X_train and X_test. So X_test are not exposed to fit() method.

In [34]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#### Building Neural Network

##### Libraries required for building ANN

In [39]:
import tensorflow as tf

##### Instantiating Sequential model

In [40]:
ann = tf.keras.models.Sequential()

##### Adding hidden layers

##### Hidden layer 1
Adding first hidden layer using rectifier activation function named 'relu'. Number of neurons is selected randomly as 6. 'Dense' method is used to create fully connected neurons.

In [42]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

##### Hidden layer 2
Adding second hidden layer using rectifier activation function named 'relu'. Number of neurons is selected randomly as 6.

In [43]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

##### Output layer
Adding output layer using sigmoid activation function named 'sigmoid'. Sigmoid is a good way to understand the output as a probability. As the output is binary, selecting units as 1.

In [44]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#### Training the Neural network
Using 'adam' optimizer which is a stochastic gradient descent optimizer and 'binary_crossentropy' cost function as the output is binary. Adding 'accuracy' into performance metrics for evaluation.

In [46]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

##### Training
Training on train data with a batch_size of 32 to compare the result with actual in batches and 100 epochs to repeat the process 100 times for the dataset.

In [47]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 3s 4ms/step - loss: 0.5848 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5098 - accuracy: 0.7960: 0s - loss:
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4746 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4510 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4383 - accuracy: 0.7960: 0s -
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4326 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4291 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4264 - accuracy: 0.7960
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4240 - accuracy: 0.7961
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - los

#### Prediction on test set

In [52]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print("Predicting that ", sum(y_pred) ," customers will exit out of ", len(y_pred) )

Predicting that  [289]  customers will exit out of  2000


#### Evaluating the model

##### Confusion matrix

In [54]:
from sklearn.metrics import confusion_matrix, accuracy_score
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[1517   78]
 [ 194  211]]


##### Accuracy

In [58]:
"Accuracy is {:.0%}".format(accuracy_score(y_test, y_pred))    

'Accuracy is 86%'